In [ ]:
import pandas as pd 
from pandas import Series,DataFrame
import os , random , sys, time
from urllib.parse import urlparse
from selenium import webdriver 
from bs4 import BeautifulSoup


In [ ]:
browser = webdriver.Chrome('chromedriver.exe')

## LongIn

In [ ]:
browser.get('https://www.linkedin.com/uas/login')

In [ ]:
file = open ('user-passw.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [ ]:
elementID = browser.find_element_by_id("username")
elementID.send_keys(username)

In [ ]:
elementID = browser.find_element_by_id("password")
elementID.send_keys(password)

In [ ]:
elementID.submit()

## Profiles Links

In [ ]:
file_profile = open ('profileLinks.txt')
profilelLinks = []

for visitProfile in file_profile :
    fullLink =  visitProfile 
    profilelLinks.append(fullLink)

print(profilelLinks)

## LinkedIn

In [ ]:
dic={}
df = DataFrame(columns = ['NAME', 'COMPANY' , 'JOB_TITLE','LOCATION','PROFILE_TITLE', 'LINK'])

for link in profilelLinks:

    browser.get(link)
    scroll_pause_time = 5
    
    #get scroll height 
    last_height = browser.execute_script('return document.body.scrollHeight')
    time.sleep(scroll_pause_time)
    
    for i in range(2):
        #scroll down to bottom
        #browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scheight = .1
        while scheight < 9.9:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight/%s);" % scheight)
            scheight += .01
            

        #wait to load page 
       # time.sleep(scroll_pause_time)

        #calculate new scroll height and compare with last scroll height 
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        las_height = new_height
    
    src = browser.page_source
    soup = BeautifulSoup(src,'html5lib')
    
    #name
    name_div= soup.find('div',{'class':'flex-1 mr5'} )
    name_loc = name_div.find_all('ul')
    name = name_loc[0].find('li').get_text().strip()
    print(name)
    
    
    #loc
    loc = name_loc[1].find('li').get_text().strip()
    print(loc)
    
    
    #Profile_title
    profile_title = name_div.find('h2').get_text().strip()
    print(profile_title)

   
    #job title-1(Profiles with one possition in the company )
    exp_section = soup.find('section' , {'id':"experience-section"})
    exp_ul = exp_section.find('ul')
    li_tags = exp_ul.find('li' , {"class" :"pv-entity__position-group-pager pv-profile-section__list-item ember-view"})
    a_tags = li_tags.find('a')
    job_title = a_tags.find('h3').get_text().strip()
    
    
    #job title-2 (Profile with Different position in the same company)
    if 'Company Name' in job_title or 'Nome da empresa' in job_title :
       
        ul_tags = li_tags.find('ul')
        h3_tags = ul_tags.find('h3')
        span_tag= h3_tags.find_all('span')
        job_title = span_tag[1].get_text().strip()
        print(job_title)
        
       
        #Company name-2 (Profile with different position in the same company)
        span_tags2 = li_tags.find('h3')
        company_name= span_tags2.find_all('span')
        company_name = company_name[1].get_text().strip() 
        
        
        #Company name-1 (Profiles with same position in the same company)
    else:
        print(job_title)
        
        company_name= a_tags.find_all('p')[1].get_text().strip().split('\n') #there is 2 paragraf , the second one will be selected 
        company_name = company_name[0]
        print(company_name)
        
    
    dic ={'NAME' :name,'COMPANY' :company_name, 'JOB_TITLE' :job_title,'LOCATION':loc,'PROFILE_TITLE':profile_title,  'LINK':fullLink }
    
    
    df=df.append(dic,ignore_index=True)
    

df.head(20)
    
    
    


In [ ]:
df.to_csv('C:\\yyy\\yyyy\\yyyy\\yyy\\yyy\\yyyyy' , sep= ';')

## Sales Navigator

In [ ]:
dic={}
df = DataFrame(columns = ['NAME', 'COMPANY' , 'JOB_TITLE','LOCATION','PROFILE_TITLE', 'LINK'])

for link in profilelLinks:
    
    browser.get(link)
    scroll_pause_time = 5

    #get scroll height 
    last_height = browser.execute_script('return document.body.scrollHeight')
    time.sleep(scroll_pause_time)

    for i in range(1):
    #scroll down to bottom
    #browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scheight = .1
        while scheight < 9.9:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight/%s);" % scheight)
            scheight += .01


        #wait to load page 
        # time.sleep(scroll_pause_time)

        #calculate new scroll height and compare with last scroll height 
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        las_height = new_height

    src = browser.page_source
    soup = BeautifulSoup(src,'html5lib')

    #name
    name_div = soup.find('div',{'class':"profile-topcard full-width pb5"} )
    name_dl = name_div.find('dl',{'class':"profile-topcard-person-entity__content-text vertical-align-top pl4"}) 
    name_dt = name_dl.find('dt' , {'class':'flex align-items-center'})
    name = name_dt.find('span').get_text().strip()

    print(name)

    #loc

    loc_dl = name_dl.find('dd', {'class' : 'mt4 mb0'})
    loc = name_dl.find('div' , {'class' : "profile-topcard__location-data inline t-14 t-black--light mr5"}).get_text().strip()
    print(loc)

    #Profile_title
    profile_title = name_dl.find('dd',{'class': 'mt2'}).get_text().strip()
    print(profile_title)

    #job title

    exp_dl = name_div.find('dl' , {'class' :'overflow-hidden'})
    exp_dd = exp_dl.find('dd' , {'class' :'profile-topcard__current-positions flex mt3'})
    exp_div = exp_dd.find('div',{'class' :'profile-topcard__summary-position flex align-items-top' })
    job_title = exp_div.find('span',{'class' :"profile-topcard__summary-position-title" }).get_text().strip()

    print(job_title)

    company_span = exp_div.find('span', {'class' : 'align-self-center'})
    
    company_name = company_span.find('a')
    
    if company_name == None:
        
        company_name == None
        
        print(company_name)
    else:
        company_name = company_name.get_text().strip()
        
        print(company_name)
        


    #DataFrame
    dic ={'NAME' :name,'COMPANY' :company_name, 'JOB_TITLE' :job_title,'LOCATION':loc,'PROFILE_TITLE':profile_title,  'LINK':link }

    df=df.append(dic,ignore_index=True)


df.head(20)
    
    
    


# Saving dfProfiles

In [ ]:
df.to_csv('C:\\xxxx\\xxxx\\xxxxx\\xxxx\\xxxx\\xxxx\\xxxx' , sep= ';')